In [1]:
#Define functions to extract features from images and test a method
#Always run this first cell for all the imports and global variable definitions
#To be run in the workenvAutoencoder2 environment

from skimage.feature import graycomatrix, graycoprops
import numpy as np
from tqdm import tqdm
import os

import pywt
from skimage.io import imread
from skimage.color import rgb2gray

from skimage.feature import local_binary_pattern
from skimage.util import img_as_ubyte
from skimage.transform import radon
from scipy.ndimage import gaussian_filter
from scipy.ndimage import generic_filter
from scipy.stats import kurtosis, skew
import nibabel as nib
import SimpleITK as sitk
import mahotas as mh
import tifffile as tif
import xgboost as xgb

from radiomics import featureextractor

MAIN_DIR = 'E:/AAV para enfermedades renales/LSFM combined images/TextureAnalysis/Tiles3D/CenteredInGlom-prepared'


def normalize_image(image):
    # Normalize the image to the range [0, 1]
    image = image - np.min(image)
    image = image / np.max(image)
    return image


#Define several functions to extract features from an image
#-----------------------------------------------------------------------------------------

#General Features
def calculate_statistical_features(image):
    mean_val = np.mean(image)
    variance_val = np.var(image)
    skewness_val = skew(image,axis=None)
    kurtosis_val = kurtosis(image,axis=None)
    #hist, _ = np.histogram(image, bins=256, range=(0, 255))
    #hist = hist / np.sum(hist)
    #entropy_val = -np.sum(hist * np.log2(hist + np.finfo(float).eps))
    return mean_val, variance_val, skewness_val, kurtosis_val
    
#Wavelet features in 3D
def calculate_wavelet_features3D(image):
    # Perform 3D discrete wavelet transform
    coeffs = pywt.dwtn(image, 'db1')
    
    # Extract approximation and detail coefficients
    cA = coeffs['aaa']
    cH = coeffs['daa']
    cV = coeffs['ada']
    cD = coeffs['aad']
    
    # Compute the mean of the coefficients
    approx_coeff_mean = np.mean(cA)
    horiz_coeff_mean = np.mean(cH)
    vert_coeff_mean = np.mean(cV)
    depth_coeff_mean = np.mean(cD)

    return approx_coeff_mean, horiz_coeff_mean, vert_coeff_mean, depth_coeff_mean

    
#Fourier features in 3D
def calculate_fourier_features_3d(image):
    # Perform 3D Fast Fourier Transform
    fft_image = np.fft.fftn(image)
    fft_magnitude = np.abs(fft_image)

    # Find the dominant frequency
    dominant_frequency_magnitude = np.max(fft_magnitude)
    
    # Calculate the total energy
    total_energy = np.sum(fft_magnitude ** 2)

    return dominant_frequency_magnitude, total_energy
    
#Local Binary Patterns (LBP) uniformity in 3D
def lbp_code_3d(neighborhood):
    center = neighborhood[len(neighborhood) // 2]
    binary_pattern = (neighborhood >= center).astype(int)
    binary_pattern = np.delete(binary_pattern, len(binary_pattern) // 2)  # Remove the center pixel
    lbp_value = np.dot(binary_pattern, 1 << np.arange(binary_pattern.size)[::-1])
    return lbp_value

def calculate_lbp_uniformity_3d(image, radius=1):
    # Define the neighborhood shape for 3D LBP
    neighborhood_shape = (2 * radius + 1, 2 * radius + 1, 2 * radius + 1)
    
    # Apply the 3D LBP calculation
    lbp_image = generic_filter(image, lbp_code_3d, size=neighborhood_shape, mode='constant', cval=0)
    
    # Compute the histogram of LBP values
    lbp_hist, _ = np.histogram(lbp_image.ravel(), bins=np.arange(0, 2**(neighborhood_shape[0]**3 - 1) + 1), range=(0, 2**(neighborhood_shape[0]**3 - 1)))
    lbp_hist = lbp_hist / np.sum(lbp_hist)  # Normalize histogram
    
    # Calculate uniformity measure
    lbp_uniformity = np.sum(lbp_hist**2)

    return lbp_uniformity


def calculate_porosity(image):
    threshold = np.mean(image)  # Use mean intensity as a threshold
    hole_area = np.sum(image < threshold)  # Count low-intensity pixels (holes)
    total_area = image.size  # Total number of pixels
    porosity = hole_area / total_area
    return porosity
#-----------------------------------------------------------------------------------------

#Define a function to extract all features from a set of images and produce a numpy array
def extract_features(MAIN_DIR,setName):
    # Define GLCM parameters
    featureNames = 'mean_val', 'variance_val', 'skewness_val', 'kurtosis_val',\
    'approx_coeff_mean', 'horiz_coeff_mean', 'vert_coeff_mean', 'depth_coeff_mean',\
    'dominant_frequency_magnitude', 'total_energy',\
    'lbp_uniformity',\
    'porosity'
    
    initDir = os.getcwd()

    os.chdir(MAIN_DIR)

    #Entry 0 to store the Healthy features, entry 1 to store the Pathological features
    data = [[], []]

    print(f'Processing images from the {setName} set:')

    for stateName in os.listdir(f'{setName}'):
        #For the Healthy and Pathological folders
        if stateName == 'Healthy':
            stateNum = 0
            
        elif stateName == 'Pathological':
            stateNum = 1
            
        print(f'{stateName} images')

        for imName in tqdm(os.listdir(f'{setName}/{stateName}')):
                
            # Load and normalize the image
            image = tif.imread(f'{setName}/{stateName}/{imName}')
            image = normalize_image(image)

            # Statistical features
            mean_val, variance_val, skewness_val, kurtosis_val = calculate_statistical_features(image)         
            
            # Wavelet features
            approx_coeff_mean, horiz_coeff_mean, vert_coeff_mean, depth_coeff_mean = calculate_wavelet_features3D(image)

            # Fourier features
            dominant_frequency_magnitude, total_energy = calculate_fourier_features_3d(image)

            # Average diameter of circles with radon transform
            #average_diameter_radon,average_radial_intensity_radon = measure_average_diameter_radon(image)

            #LBP uniformity
            lbp_uniformity = calculate_lbp_uniformity_3d(image)

            #Porosity
            porosity = calculate_porosity(image)

            features = np.array([mean_val, variance_val, skewness_val, kurtosis_val,  
                                 approx_coeff_mean, horiz_coeff_mean, vert_coeff_mean, depth_coeff_mean,
                                 dominant_frequency_magnitude, total_energy, 
                                 lbp_uniformity,
                                 porosity])
            
            #Save the features in the corresponding list
            data[stateNum].append(features)

    healthyFeatures = np.vstack(data[0])
    pathologicalFeatures = np.vstack(data[1])

    healthyLabels = np.zeros(healthyFeatures.shape[0],dtype=int)
    pathologicalLabels = np.ones(pathologicalFeatures.shape[0],dtype=int)

    allFeatures = np.vstack([healthyFeatures, pathologicalFeatures])
    allLabels = np.hstack([healthyLabels, pathologicalLabels])

    os.chdir(initDir)
    return allFeatures, allLabels, featureNames

def get_test_results(method,testData,testLabels,dataTuple = False):
    #Test a method and print the results

    if dataTuple is True:
        #The dataformat is a tuple with the features and the labels
        testData = xgb.DMatrix(testData, label=testLabels)

    #Positives will be the Pathological images
    truePos = 0
    trueNeg = 0
    falsePos = 0
    falseNeg = 0

    predictions = method.predict(testData)

    #If the predictions are not a binary vector
    if not np.all(np.isin(predictions, [0, 1])):
        #Threshold them to make them binary
        predictions = np.where(predictions >= 0.5, 1, 0)

    print('predictions')
    print(predictions)

    print('ground truth labels')
    print(testLabels)
    
    for i in range(len(predictions)):
        if predictions[i] == testLabels[i] and testLabels[i] == 1:
            truePos += 1
        elif predictions[i] == testLabels[i] and testLabels[i] == 0:
            trueNeg += 1
        elif predictions[i] != testLabels[i] and testLabels[i] == 0:
            falsePos += 1
        elif predictions[i] != testLabels[i] and testLabels[i] == 1:
            falseNeg += 1
        #print(f'Predicted: {predictions[i]}, True: {test_labels[i]}')

    print(f'Total predictions: {len(predictions)}')
    print(f'True positives: {truePos}')
    print(f'True negatives: {trueNeg}')
    print(f'False positives: {falsePos}')
    print(f'False negatives: {falseNeg}')
    print(f'Accuracy: {(truePos+trueNeg)/len(predictions)}')
    print(f'Sensitivity: {truePos/(truePos+falseNeg)}')
    print(f'Specificity: {trueNeg/(trueNeg+falsePos)}')
    print(f'f1-score: {2*truePos/(2*truePos+falsePos+falseNeg)}')


In [ ]:
#Extract features from the images and save them in text files

testData,testLabels,featureNames = extract_features(MAIN_DIR,'test')
header = ','.join(featureNames)
np.savetxt('testData.txt', testData, delimiter=',', header=header, comments='')
np.savetxt('testLabels.txt', testLabels, delimiter=',', header='Label', comments='')

valData,valLabels,featureNames = extract_features(MAIN_DIR,'validation')
header = ','.join(featureNames)
np.savetxt('valData.txt', valData, delimiter=',', header=header, comments='')
np.savetxt('valLabels.txt', valLabels, delimiter=',', header='Label', comments='', fmt='%d')

trainData,trainLabels,featureNames = extract_features(MAIN_DIR,'train')
header = ','.join(featureNames)
np.savetxt('trainData.txt', trainData, delimiter=',', header=header, comments='')
np.savetxt('trainLabels.txt', trainLabels, delimiter=',', header='Label', comments='', fmt='%d')

In [40]:
#Read the data from the files to later apply a classifier


#First read the feature names
featureNames = np.genfromtxt('testData.txt', delimiter=',', max_rows=1, dtype=str)

#Then load the data
testData = np.loadtxt('testData.txt', delimiter=',', skiprows=1)
testLabels = np.loadtxt('testLabels.txt', delimiter=',', skiprows=1)

valData = np.loadtxt('valData.txt', delimiter=',', skiprows=1)
valLabels = np.loadtxt('valLabels.txt', delimiter=',', skiprows=1)

trainData = np.loadtxt('trainData.txt', delimiter=',', skiprows=1)
trainLabels = np.loadtxt('trainLabels.txt', delimiter=',', skiprows=1)

In [ ]:
# Fit an XGBoost classifier and test it (with XGBClassifier), after performing hyperparameter grid search
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import numpy as np
import pandas as pd
import json

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2],
    'min_child_weight': [1, 3, 5],
    'reg_alpha': [0, 0.01, 0.1],
    'reg_lambda': [1, 1.5, 2]
}

# Combine train and validation data
combinedData = np.concatenate((trainData, valData), axis=0)
combinedLabels = np.concatenate((trainLabels, valLabels), axis=0)

# Create a validation fold index
test_fold = np.concatenate((
    np.full(trainData.shape[0], -1),  # -1 for training set
    np.zeros(valData.shape[0])        # 0 for validation set
))

# Create PredefinedSplit
ps = PredefinedSplit(test_fold)

# Initialize the model
model = XGBClassifier(
    objective='binary:logistic', 
    eval_metric='logloss',
    use_label_encoder=False,  
    tree_method='gpu_hist'  
)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=ps,  # Use PredefinedSplit
    verbose=2,  # Set verbose to 2 for detailed progress information
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(combinedData, combinedLabels)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)
# Save the best parameters to a text file
with open('best_params.txt', 'w') as file:
    json.dump(best_params, file)

In [ ]:
#Load the best weights saved from a hyperparameter grid search and fit a classification model

import json
from xgboost import XGBClassifier

# Read the best parameters from the text file
with open('best_params.txt', 'r') as file:
    best_params = json.load(file)

# Initialize the model with the best parameters
model2 = XGBClassifier(
    objective='binary:logistic', 
    eval_metric='logloss',
    use_label_encoder=False,  # For compatibility
    tree_method='gpu_hist',  # Use GPU for training if available
    **best_params
)

# Fit the model using the training data
model2.fit(trainData, trainLabels, eval_set=[(valData, valLabels)], verbose=False)

# Make predictions on the test set
y_pred_prob = model2.predict_proba(testData)[:, 1]  # Probabilities
y_pred = (y_pred_prob > 0.5).astype(int)  # Convert to binary predictions

# Confusion matrix
cm = confusion_matrix(testLabels, y_pred)
tn, fp, fn, tp = cm.ravel()  # Extract TN, FP, FN, TP

# Print results
print("\nClassification Report:\n", classification_report(testLabels, y_pred))
print("\nConfusion Matrix:")
print(cm)
print(f"True Positives (TP): {tp}")
print(f"True Negatives (TN): {tn}")
print(f"False Positives (FP): {fp}")
print(f"False Negatives (FN): {fn}")

# Get feature importances
importances = model2.feature_importances_

# Create a DataFrame for feature importances
feature_importance_df = pd.DataFrame({
    'Feature': featureNames,
    'Importance': importances
})

# Sort the DataFrame by importance
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Display the feature importances
print("\nFeature Importances:")
print(feature_importance_df)